This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

vXzxFhEcDMV1s5wRREQC


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json'
r = requests.get(url)
print(dir(r))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

In [5]:
text = r.text
print(text)

{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.\u003cbr\u003e\u003cbr\u003eTrading System: Xetra\u003cbr\u003e\u003cbr\u003eISIN: DE0005313704","refreshed_at":"2020-12-01T14:48:09.907Z","newest_available_date":"2020-12-01","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":null,"transform":null,"column_index":null,"start_date":"2000-06-07","end_date":"2020-12-01","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null],["2020-11-30",111.0,113.6,111.0,112.1,null,315.0,35111.5,null,null,null],["2020-11-27",111.0,111.5,109.5,111.2,null,155.0,17168.5,null,null,null],["2020-11-26",111.3,112.0,110.5,111.6,null,254.0,2

In [6]:
soup = BeautifulSoup(text)
pretty_soup = soup.prettify()
print(pretty_soup)

<html>
 <body>
  <p>
   {"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.\u003cbr\u003e\u003cbr\u003eTrading System: Xetra\u003cbr\u003e\u003cbr\u003eISIN: DE0005313704","refreshed_at":"2020-12-01T14:48:09.907Z","newest_available_date":"2020-12-01","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":null,"transform":null,"column_index":null,"start_date":"2000-06-07","end_date":"2020-12-01","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null],["2020-11-30",111.0,113.6,111.0,112.1,null,315.0,35111.5,null,null,null],["2020-11-27",111.0,111.5,109.5,111.2,null,155.0,17168.5,null,null,null],["2020-11-26",111.3,112.0,

In [7]:
json_data = r.json()
print(json.dumps(json_data, indent = 4))

{
    "dataset": {
        "id": 10095370,
        "dataset_code": "AFX_X",
        "database_code": "FSE",
        "name": "Carl Zeiss Meditec (AFX_X)",
        "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
        "refreshed_at": "2020-12-01T14:48:09.907Z",
        "newest_available_date": "2020-12-01",
        "oldest_available_date": "2000-06-07",
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "frequency": "daily",
        "type": "Time Series",
        "premium": false,
        "limit": null,
        "transform": null,
        "column_index": null,
        "start_date": "2000-06-07",
        "end_date": 

In [8]:
for k in json_data['dataset'].keys():
    print(k + ': ', json_data['dataset'][k])

id:  10095370
dataset_code:  AFX_X
database_code:  FSE
name:  Carl Zeiss Meditec (AFX_X)
description:  Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704
refreshed_at:  2020-12-01T14:48:09.907Z
newest_available_date:  2020-12-01
oldest_available_date:  2000-06-07
column_names:  ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
frequency:  daily
type:  Time Series
premium:  False
limit:  None
transform:  None
column_index:  None
start_date:  2000-06-07
end_date:  2020-12-01
data:  [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None], ['2020-11-30', 111.0, 113.6, 111.0, 112.1, None, 315.0, 35111.5, None, None, None], ['2020-11-27', 111.0, 111.5, 109.5, 111.2, None, 155.0, 17168.5, None, None, None], ['2020-11-26', 111.3, 112.0, 110.5, 111.6, None, 254.0, 28367.5, None, None, None

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
columns = json_data['dataset']['column_names']
data = json_data['dataset']['data']
dataframe = pd.DataFrame(data, columns = columns)
dataframe

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2020-12-01,112.2,112.2,111.5,112.0,NaN,51.0,5703.0,None,None,NaN
1,2020-11-30,111.0,113.6,111.0,112.1,NaN,315.0,35111.5,None,None,NaN
2,2020-11-27,111.0,111.5,109.5,111.2,NaN,155.0,17168.5,None,None,NaN
3,2020-11-26,111.3,112.0,110.5,111.6,NaN,254.0,28367.5,None,None,NaN
4,2020-11-25,111.4,111.4,110.1,110.1,NaN,294.0,32435.8,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5263,2000-06-13,NaN,NaN,NaN,45.0,NaN,0.0,NaN,None,None,0.0
5264,2000-06-12,NaN,NaN,NaN,45.0,NaN,0.0,NaN,None,None,0.0
5265,2000-06-09,NaN,NaN,NaN,45.0,NaN,0.0,NaN,None,None,0.0
5266,2000-06-08,NaN,NaN,NaN,38.0,NaN,0.0,NaN,None,None,0.0


In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   5268 non-null   object 
 1   Open                   4536 non-null   float64
 2   High                   4867 non-null   float64
 3   Low                    4867 non-null   float64
 4   Close                  5268 non-null   float64
 5   Change                 9 non-null      float64
 6   Traded Volume          5222 non-null   float64
 7   Turnover               3913 non-null   float64
 8   Last Price of the Day  0 non-null      object 
 9   Daily Traded Units     0 non-null      object 
 10  Daily Turnover         1309 non-null   float64
dtypes: float64(8), object(3)
memory usage: 452.8+ KB


In [11]:
dataframe['Date'] = pd.to_datetime(dataframe['Date'], format = '%Y-%m-%d')
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   5268 non-null   datetime64[ns]
 1   Open                   4536 non-null   float64       
 2   High                   4867 non-null   float64       
 3   Low                    4867 non-null   float64       
 4   Close                  5268 non-null   float64       
 5   Change                 9 non-null      float64       
 6   Traded Volume          5222 non-null   float64       
 7   Turnover               3913 non-null   float64       
 8   Last Price of the Day  0 non-null      object        
 9   Daily Traded Units     0 non-null      object        
 10  Daily Turnover         1309 non-null   float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 452.8+ KB


In [12]:
df = dataframe[(dataframe['Date'] >= '2017-01-01') & (dataframe['Date'] <= '2017-12-31')]
df

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
727,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,NaN
728,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,NaN
729,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,NaN
730,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0,3641949.0,None,None,NaN
731,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0,6179433.0,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
977,2017-01-06,34.91,35.21,34.91,35.04,NaN,27507.0,964046.0,None,None,NaN
978,2017-01-05,35.02,35.20,34.73,35.06,NaN,48412.0,1692326.0,None,None,NaN
979,2017-01-04,35.48,35.51,34.75,35.19,NaN,54408.0,1906810.0,None,None,NaN
980,2017-01-03,35.90,35.93,35.34,35.48,NaN,70618.0,2515473.0,None,None,NaN


In [13]:
q3_max = round(df['Open'].max(), 2)
q3_min = round(df['Open'].min(), 2)

In [14]:
df['change'] = round(df['Close'] - df['Open'],2)
df['abs_change'] = df['change'].abs()
q4 = df.loc[df['abs_change'] == df['abs_change'].max(), 'change'].iloc[0]

C:\Users\jacob\AppData\Local\Temp\ipykernel_27020\3164703929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['change'] = round(df['Close'] - df['Open'],2)
C:\Users\jacob\AppData\Local\Temp\ipykernel_27020\3164703929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abs_change'] = df['change'].abs()


In [15]:
close = df['Close'].values
matrix = close[:, None] - close
if abs(matrix).max() > matrix.max():
    q5 = abs(matrix).max() * (-1)
else:
    q5 = matrix.max()
q5

19.03

In [16]:
q6 = round(df['Traded Volume'].mean(), 2)

In [17]:
q7 = round(df['Traded Volume'].median(), 2)

In [18]:
print(f'Question 3: The highest opening price in 2017 was ${q3_max}. The lowest opening price in 2017 was ${q3_min}.')
print(f'Question 4: The largest change in any one day was ${q4}.')
print(f"Question 5: The largest change between any two days' closing price was ${q5}.")
print(f'Question 6: The average daily trading volume during this year was ${q6}')
print(f'Question 7: The median daily trading volume during this year was ${q7}')

Question 3: The highest opening price in 2017 was $53.11. The lowest opening price in 2017 was $34.0.
Question 4: The largest change in any one day was $-1.69.
Question 5: The largest change between any two days' closing price was $19.03.
Question 6: The average daily trading volume during this year was $89124.34
Question 7: The median daily trading volume during this year was $76286.0
